## Question 1. Understanding Docker images
### Run docker with the python:3.13 image. Use an entrypoint bash to interact with the container.

### What's the version of pip in the image?

In [ ]:
# docker run -it --rm --entrypoint=bash python:3.13
# pip --version

## Prepare the Data

In [12]:
import pandas as pd
import pyarrow as pa

In [14]:
import pyarrow.parquet as pq

In [19]:
df_taxi = pq.read_table('green_tripdata_2025-11.parquet')
df_taxi = df_taxi.to_pandas()

In [20]:
df_zones = pd.read_csv('taxi_zone_lookup.csv')

In [22]:
df_taxi.info()

<class 'pandas.DataFrame'>
RangeIndex: 46912 entries, 0 to 46911
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               46912 non-null  int32         
 1   lpep_pickup_datetime   46912 non-null  datetime64[us]
 2   lpep_dropoff_datetime  46912 non-null  datetime64[us]
 3   store_and_fwd_flag     41343 non-null  str           
 4   RatecodeID             41343 non-null  float64       
 5   PULocationID           46912 non-null  int32         
 6   DOLocationID           46912 non-null  int32         
 7   passenger_count        41343 non-null  float64       
 8   trip_distance          46912 non-null  float64       
 9   fare_amount            46912 non-null  float64       
 10  extra                  46912 non-null  float64       
 11  mta_tax                46912 non-null  float64       
 12  tip_amount             46912 non-null  float64       
 13  tolls_amount

## Q3. For the trips in November 2025 (lpep_pickup_datetime between '2025-11-01' and '2025-12-01', exclusive of the upper bound), how many trips had a trip_distance of less than or equal to 1 mile?

In [37]:
df3 = df_taxi[['lpep_pickup_datetime','trip_distance']]

In [38]:
df3.info()

<class 'pandas.DataFrame'>
RangeIndex: 46912 entries, 0 to 46911
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   lpep_pickup_datetime  46912 non-null  datetime64[us]
 1   trip_distance         46912 non-null  float64       
dtypes: datetime64[us](1), float64(1)
memory usage: 733.1 KB


In [49]:
df_nov = df3[
    (df3['lpep_pickup_datetime'] >= '2025-11-01') &
    (df3['lpep_pickup_datetime'] < '2025-12-01')
]

In [60]:
df_nov.head()

,lpep_pickup_datetime,trip_distance
0,2025-11-01 00:34:48,0.74
1,2025-11-01 00:18:52,0.95
2,2025-11-01 01:03:14,2.19
3,2025-11-01 00:10:57,5.44
4,2025-11-01 00:03:48,3.20


In [51]:
(df_nov['trip_distance'] <= 1).sum()

np.int64(8007)

In [ ]:
# 8007

## Q4. Which was the pick up day with the longest trip distance? Only consider trips with trip_distance less than 100 miles (to exclude data errors).

### Use the pick up time for your calculations.

In [56]:
df4 = df3[(df3['trip_distance'] < 100)]

In [61]:
df4.head()

,lpep_pickup_datetime,trip_distance
0,2025-11-01 00:34:48,0.74
1,2025-11-01 00:18:52,0.95
2,2025-11-01 01:03:14,2.19
3,2025-11-01 00:10:57,5.44
4,2025-11-01 00:03:48,3.20


In [59]:
df4.loc[df4["trip_distance"].idxmax()]

lpep_pickup_datetime    2025-11-14 15:36:27
trip_distance                         88.03
Name: 18867, dtype: object

In [ ]:
# lpep_pickup_datetime    2025-11-14 

## Q5. Which was the pickup zone with the largest total_amount (sum of all trips) on November 18th, 2025?

In [63]:
df_full = df_taxi.merge(df_zones, left_on='PULocationID', right_on='LocationID', how='left')

In [83]:
df5 = df_full[['lpep_pickup_datetime','Zone','total_amount']]

In [84]:
df5["pickup_day"] = df5["lpep_pickup_datetime"].dt.floor("D")

In [85]:
df18_11 = df5[(df5['pickup_day'] == '2025-11-18')]

In [88]:
df18_11.groupby("Zone", as_index=False)["total_amount"].sum().sort_values("total_amount", ascending=False)

,Zone,total_amount
39,East Harlem North,9281.92
40,East Harlem South,6696.13
23,Central Park,2378.79
128,Washington Heights South,2139.05
85,Morningside Heights,2100.59
...,...,...
125,Upper West Side South,16.00
124,Upper West Side North,16.00
129,West Village,16.00
12,Bronxdale,13.57


In [ ]:
# East Harlem North	9281.92

## Q6. For the passengers picked up in the zone named "East Harlem North" in November 2025, which was the drop off zone that had the largest tip?

### Note: it's tip , not trip. We need the name of the zone, not the ID.

In [101]:
df6 = df_full[['lpep_pickup_datetime','Zone','tip_amount','DOLocationID']]
df6["pickup_day"] = df6["lpep_pickup_datetime"].dt.floor("D")

In [107]:
df6_pu_ehn_nov = df6[(df6['Zone']=='East Harlem North') &
    (
        (df6['lpep_pickup_datetime'] >= '2025-11-01') &
        (df6['lpep_pickup_datetime'] < '2025-12-01')
    )]

In [108]:
df6_pu_ehn_nov.sort_values('lpep_pickup_datetime')

,lpep_pickup_datetime,Zone,tip_amount,DOLocationID,pickup_day
1,2025-11-01 00:18:52,East Harlem North,0.00,42,2025-11-01
68,2025-11-01 00:20:37,East Harlem North,0.00,41,2025-11-01
53,2025-11-01 00:20:44,East Harlem North,0.00,74,2025-11-01
69,2025-11-01 00:33:46,East Harlem North,3.12,74,2025-11-01
0,2025-11-01 00:34:48,East Harlem North,1.94,42,2025-11-01
...,...,...,...,...,...
41322,2025-11-30 22:30:18,East Harlem North,5.02,243,2025-11-30
41316,2025-11-30 22:37:49,East Harlem North,0.00,262,2025-11-30
41321,2025-11-30 22:38:13,East Harlem North,3.19,263,2025-11-30
41333,2025-11-30 23:29:33,East Harlem North,0.00,243,2025-11-30


In [110]:
q6 = df6_pu_ehn_nov[['tip_amount','DOLocationID']].merge(df_zones, left_on='DOLocationID', right_on='LocationID', how='left')

In [114]:
q6[['tip_amount','Zone']].sort_values('tip_amount')

,tip_amount,Zone
2282,0.00,Midtown Center
2242,0.00,East Harlem South
5933,0.00,Hamilton Heights
5932,0.00,Central Harlem North
8103,0.00,East Harlem North
...,...,...
5513,28.90,NaN
9087,34.25,Long Island City/Queens Plaza
10842,45.00,East Harlem North
8729,50.00,LaGuardia Airport


In [ ]:
# 81.89	Yorkville West